**Name: Anjitha Merin Alex**
    
**Rollno: D21005**

### <a id='11'> Index </a>

* [Content Based Recommender system](#12)

* [Collaborative Filtering Recommender System](#13)

<a id = "12">Content Based Recommender system</a>

### Content Based Recommender system, in this notebook uses two new methods:


[1.Standardization of ratings:](#0)

We have performed stanardization of ratings before recommending movies,to avoid bias from tough raters and easy raters

[2.Created a function to find user preference based on recency(found from timestamp data)
user_preference_withdate(userid, top_n =5):](#1)

The function takes parameters userid and number of top_n movies the user prefers, and returns user preferred movies based on timestamp

Approach Taken:
    
    1. Create a movie-Genre binary matrix by giving value 1 for genres present in a particular movie
    
    2. Create movie - movie similarity matrix
    
    3. Get top_n movies for a particular user using function user_preference
    
    4. similar_movies is a function to find similar movies, this function takes movieid and minimum similarity value 
       and returns list of similar movies based on similarity score
        
    5. Find the user watchlist
    
    6. Get the movies to recommend by subtracting the user_watched movie list from similar movies list
    
    7. user_preference_withdate is a function to get top_n movies for a particular user after sorting 
      the movies on recency based on timestamp

In [1]:
#importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

### Read the data

In [2]:
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')

In [3]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
#Creating a copy of ratings dataframe to do date conversion from timestamp
df_rating_date = df_ratings.copy()


In [6]:
#date conversion from timestamp data and stored in df_rating_date dataframe
for i in range(df_ratings.shape[0]):
    df_rating_date["timestamp"][i] = datetime.fromtimestamp(df_ratings["timestamp"][i]).strftime("%d-%m-%Y")

<ipython-input-6-c7d6cfaee099>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rating_date["timestamp"][i] = datetime.fromtimestamp(df_ratings["timestamp"][i]).strftime("%d-%m-%Y")
C:\Users\anjit\anaconda\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
#Testing the code by sorting based on date
df_rating_date.sort_values("timestamp",ascending = False).head()

,userId,movieId,rating,timestamp
43899,292,102407,2.0,31-12-2016
43824,292,70286,2.0,31-12-2016
43564,292,1278,4.0,31-12-2016
43515,292,440,3.5,31-12-2016
43603,292,2291,2.5,31-12-2016


In [8]:
#find the unique ratings, to find the max ratings given
df_ratings['rating'].sort_values().unique()

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [9]:
# test code to filter out the movies which the user 1 has liked
df_ratings.loc[(df_ratings['rating'] == 5.0) & (df_ratings['userId']== 1),:]

,userId,movieId,rating,timestamp
3,1,47,5.0,964983815
4,1,50,5.0,964982931
6,1,101,5.0,964980868
8,1,151,5.0,964984041
9,1,157,5.0,964984100
...,...,...,...,...
223,1,3702,5.0,964983737
224,1,3703,5.0,964981909
225,1,3729,5.0,964982363
228,1,3793,5.0,964981855


### Data Preparation

#### Movie-Genre Binary Matrix

In [10]:
#Split genres and append into list
#df_movies.head()
#creating a dataframe with only genres, to later user this dataframe to split the individual genres
df_genre_tmp = df_movies.drop(['movieId','title'], axis = 1).copy()
genre_list = []
for row in range(len(df_genre_tmp)):
    #split the genres by the delimiter
    genre_split = df_genre_tmp.loc[row,:][0].split('|')
    for x in genre_split:
        if x not in genre_list:
            genre_list.append(x)

In [11]:
print(genre_list)

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'War', 'Musical', 'Documentary', 'IMAX', 'Western', 'Film-Noir', '(no genres listed)']


In [12]:
#Create individual columns for genre in df_movies and initialize to zero
for genre in genre_list:
    df_movies[genre] = 0

In [13]:
df_movies.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Give value 1 for genres present in a particular movie
for index, row in df_movies.iterrows():
    for genre in genre_list:
        if genre in row['genres']:
            df_movies.loc[index,genre] = 1
    

In [15]:
df_movies.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Creating a dataframe with only the genres
df_movies_genres = df_movies.iloc[:,3:].copy()

In [17]:
df_movies_genres.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
#Create movie - movie similarity matrix
cossim_matrix = pd.DataFrame(cosine_similarity(df_movies_genres))

In [19]:
cossim_matrix

,0,1,2,3,4,5,6,7,8,9,...,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741
0,1.000000,0.774597,0.316228,0.258199,0.447214,0.000000,0.316228,0.632456,0.000000,0.258199,...,0.447214,0.316228,0.316228,0.447214,0.0,0.670820,0.774597,0.00000,0.316228,0.447214
1,0.774597,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.816497,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.0,0.288675,0.333333,0.00000,0.000000,0.000000
2,0.316228,0.000000,1.000000,0.816497,0.707107,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.353553,0.000000,0.500000,0.000000,0.0,0.353553,0.408248,0.00000,0.000000,0.707107
3,0.258199,0.000000,0.816497,1.000000,0.577350,0.000000,0.816497,0.000000,0.000000,0.000000,...,0.288675,0.408248,0.816497,0.000000,0.0,0.288675,0.333333,0.57735,0.000000,0.577350
4,0.447214,0.000000,0.707107,0.577350,1.000000,0.000000,0.707107,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.707107,0.000000,0.0,0.500000,0.577350,0.00000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.670820,0.288675,0.353553,0.288675,0.500000,0.288675,0.353553,0.000000,0.500000,0.288675,...,0.750000,0.353553,0.353553,0.500000,0.0,1.000000,0.866025,0.00000,0.707107,0.500000
9738,0.774597,0.333333,0.408248,0.333333,0.577350,0.000000,0.408248,0.000000,0.000000,0.000000,...,0.577350,0.408248,0.408248,0.577350,0.0,0.866025,1.000000,0.00000,0.408248,0.577350
9739,0.000000,0.000000,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.707107,0.707107,0.000000,0.0,0.000000,0.000000,1.00000,0.000000,0.000000
9740,0.316228,0.000000,0.000000,0.000000,0.000000,0.408248,0.000000,0.000000,0.707107,0.408248,...,0.707107,0.500000,0.000000,0.707107,0.0,0.707107,0.408248,0.00000,1.000000,0.000000


In [20]:
#initialize the index and column names with the movie title
cossim_matrix.index = df_movies['title']

cossim_matrix.columns= df_movies['title']

In [21]:
#Checking for the memory space used by the cosime similarity matrix
cossim_matrix.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 9742 entries, Toy Story (1995) to Andrew Dice Clay: Dice Rules (1991)
Columns: 9742 entries, Toy Story (1995) to Andrew Dice Clay: Dice Rules (1991)
dtypes: float64(9742)
memory usage: 724.9 MB


In [22]:
cossim_matrix

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),1.000000,0.774597,0.316228,0.258199,0.447214,0.000000,0.316228,0.632456,0.000000,0.258199,...,0.447214,0.316228,0.316228,0.447214,0.0,0.670820,0.774597,0.00000,0.316228,0.447214
Jumanji (1995),0.774597,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.816497,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.0,0.288675,0.333333,0.00000,0.000000,0.000000
Grumpier Old Men (1995),0.316228,0.000000,1.000000,0.816497,0.707107,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.353553,0.000000,0.500000,0.000000,0.0,0.353553,0.408248,0.00000,0.000000,0.707107
Waiting to Exhale (1995),0.258199,0.000000,0.816497,1.000000,0.577350,0.000000,0.816497,0.000000,0.000000,0.000000,...,0.288675,0.408248,0.816497,0.000000,0.0,0.288675,0.333333,0.57735,0.000000,0.577350
Father of the Bride Part II (1995),0.447214,0.000000,0.707107,0.577350,1.000000,0.000000,0.707107,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.707107,0.000000,0.0,0.500000,0.577350,0.00000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Black Butler: Book of the Atlantic (2017),0.670820,0.288675,0.353553,0.288675,0.500000,0.288675,0.353553,0.000000,0.500000,0.288675,...,0.750000,0.353553,0.353553,0.500000,0.0,1.000000,0.866025,0.00000,0.707107,0.500000
No Game No Life: Zero (2017),0.774597,0.333333,0.408248,0.333333,0.577350,0.000000,0.408248,0.000000,0.000000,0.000000,...,0.577350,0.408248,0.408248,0.577350,0.0,0.866025,1.000000,0.00000,0.408248,0.577350
Flint (2017),0.000000,0.000000,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.707107,0.707107,0.000000,0.0,0.000000,0.000000,1.00000,0.000000,0.000000


In [23]:
#creating a movieid and title data frame
df_movieid_title = df_movies.loc[:,['movieId','title']]

### Finding User Preference

In [24]:
#Creating a copy of the ratings matrix to standardize the ratings values, to avoid the bias from tough raters and easy raters
df_ratings_std = df_ratings.copy()

In [25]:
df_ratings_std.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


<a id = 0>Standardization of Ratings </a>

In [26]:
#Standardization of rating values
df_ratings_std['rating'] = (df_ratings_std['rating']-df_ratings_std['rating'].mean())/df_ratings_std['rating'].std()

In [27]:
#Standardization of rating values for the date converted dataframe df_rating_date
df_rating_date['rating'] = (df_rating_date['rating']-df_rating_date['rating'].mean())/df_rating_date['rating'].std()

In [28]:
# finding the top 2 ratings from the standardized rating values
top_ratings = df_ratings_std['rating'].sort_values(ascending = False).unique()[0:2]

In [29]:
#store the highest and second-highest rating values
highest, second_highest = top_ratings

In [30]:
print(highest)
print(second_highest)

1.4373150989351091
0.9577122434309606


In [31]:
#function to get top_n movies for a particular user
def user_preference(userid, top_n =5):
    #store the highest and second highest rated movie list for a particular user
    movieid_5 = df_ratings_std.loc[(df_ratings_std['userId'] == userid) & (df_ratings_std['rating'] == highest),"movieId"]
    movieid_4 = df_ratings_std.loc[(df_ratings_std['userId'] == userid) & (df_ratings_std['rating'] == second_highest),"movieId"]
    
    #converting to list
    movieid_5.tolist()
    movieid_4.tolist()
    
    #If the length of the highest rated movie list is more than top_n , then randomly choose top_n movies
    if len(movieid_5) > top_n:
        return(np.random.choice(movieid_5,5))
    #Else If the length of the highest rated movie list plus the second highest rated movie list
    #is more than top_n , then choose all the movies in highest list and the remaining movies from second highest list
    elif (len(movieid_5) + len(movieid_4)) > top_n:
        return(movieid_5 + np.random.choice(movieid_4,top_n-len(movieid_5)))
    #Else return all movies from highest rated and second highest rated list
    elif (len(movieid_5) + len(movieid_4)) < top_n:
        return(movieid_5 + movieid_4)
    else:
        print('Not enough high ratings to determine user preference!')
        return(None)
    
    
    

In [32]:

#function call for user_prefence which takes a userid and return top_n movies rated by user, top_n = 5 by default
user_preference(1)

array([1732, 1517, 2700, 1049, 2141], dtype=int64)

In [33]:
#function to find similar movies, this function takes movieid
#and minimum similarity value and returns list of similar movies based on similarity score
def similar_movies(movieid, min_sim = 0.9):
    #get the movie title for the movieid
    movie_title = df_movieid_title.loc[(df_movieid_title.movieId == 1),'title'].item()
    #Extract row from similarity matrix
    similarity = cossim_matrix.loc[movie_title,:]
    #scores of all movies
    scores = np.array(similarity)
    #Related movies
    related = cossim_matrix.loc[movie_title,:][cossim_matrix.loc[movie_title,:]>min_sim].index
    return(list(related))
    
    

### Movie Recommendation

In [34]:
#Get user watchlist

In [35]:
#this function takes as input userid and returns the list of movies watched by the user
def user_watchlist(userid):
    #get the movieids watched by user
    watchlist = df_ratings["movieId"][df_ratings['userId'] == userid]
    watchlist.tolist()
    mov_names = []
    #for each of the ids append the title of the movie to mov_names list
    for id in watchlist:
        mov_names.append(df_movieid_title["title"][df_movieid_title["movieId"] == id].item())
    return(mov_names)

In [36]:
user_watchlist(1)

['Toy Story (1995)',
 'Grumpier Old Men (1995)',
 'Heat (1995)',
 'Seven (a.k.a. Se7en) (1995)',
 'Usual Suspects, The (1995)',
 'From Dusk Till Dawn (1996)',
 'Bottle Rocket (1996)',
 'Braveheart (1995)',
 'Rob Roy (1995)',
 'Canadian Bacon (1995)',
 'Desperado (1995)',
 'Billy Madison (1995)',
 'Clerks (1994)',
 'Dumb & Dumber (Dumb and Dumber) (1994)',
 'Ed Wood (1994)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Pulp Fiction (1994)',
 'Stargate (1994)',
 'Tommy Boy (1995)',
 'Clear and Present Danger (1994)',
 'Forrest Gump (1994)',
 'Jungle Book, The (1994)',
 'Mask, The (1994)',
 'Blown Away (1994)',
 'Dazed and Confused (1993)',
 'Fugitive, The (1993)',
 'Jurassic Park (1993)',
 'Mrs. Doubtfire (1993)',
 "Schindler's List (1993)",
 'So I Married an Axe Murderer (1993)',
 'Three Musketeers, The (1993)',
 'Tombstone (1993)',
 'Dances with Wolves (1990)',
 'Batman (1989)',
 'Silence of the Lambs, The (1991)',
 'Pinocchio (1940)',
 'Fargo (1996)',
 'Mission: Impossible (1996)',

### Content Based Recommender System

In [37]:
#For a given user get the list of top_n most preferred movie
userid = 1
list_preferred_movieid = user_preference(userid)

In [38]:
#For each of the movies in the above list, get similar movies
movie_list = []
for movieid in list_preferred_movieid:
    movie_list.append(similar_movies(movieid))
#the appended movie_list has sublist, the below loop flatens the list
similar_movie_list = [val for sublist in movie_list for val in sublist]
    
    

In [39]:
similar_movie_list

['Toy Story (1995)',
 'Space Jam (1996)',
 'Antz (1998)',
 'Toy Story 2 (1999)',
 'Adventures of Rocky and Bullwinkle, The (2000)',
 "Emperor's New Groove, The (2000)",
 'Shrek (2001)',
 'Monsters, Inc. (2001)',
 "Twelve Tasks of Asterix, The (Les douze travaux d'Astérix) (1976)",
 'Valiant (2005)',
 'Wild, The (2006)',
 'Ant Bully, The (2006)',
 'TMNT (Teenage Mutant Ninja Turtles) (2007)',
 'Shrek the Third (2007)',
 'Tale of Despereaux, The (2008)',
 'Toy Story 3 (2010)',
 'Shrek Forever After (a.k.a. Shrek: The Final Chapter) (2010)',
 'Gnomeo & Juliet (2011)',
 'Asterix and the Vikings (Astérix et les Vikings) (2006)',
 'Puss in Boots (Nagagutsu o haita neko) (1969)',
 'Turbo (2013)',
 'The Lego Movie (2014)',
 'Home (2015)',
 'Inside Out (2015)',
 'The Good Dinosaur (2015)',
 'Moana (2016)',
 'Toy Story (1995)',
 'Space Jam (1996)',
 'Antz (1998)',
 'Toy Story 2 (1999)',
 'Adventures of Rocky and Bullwinkle, The (2000)',
 "Emperor's New Groove, The (2000)",
 'Shrek (2001)',
 'Mon

In [40]:
user_watched_list = user_watchlist(userid)

In [41]:
#movies watched by a particular user
user_watched_list

['Toy Story (1995)',
 'Grumpier Old Men (1995)',
 'Heat (1995)',
 'Seven (a.k.a. Se7en) (1995)',
 'Usual Suspects, The (1995)',
 'From Dusk Till Dawn (1996)',
 'Bottle Rocket (1996)',
 'Braveheart (1995)',
 'Rob Roy (1995)',
 'Canadian Bacon (1995)',
 'Desperado (1995)',
 'Billy Madison (1995)',
 'Clerks (1994)',
 'Dumb & Dumber (Dumb and Dumber) (1994)',
 'Ed Wood (1994)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Pulp Fiction (1994)',
 'Stargate (1994)',
 'Tommy Boy (1995)',
 'Clear and Present Danger (1994)',
 'Forrest Gump (1994)',
 'Jungle Book, The (1994)',
 'Mask, The (1994)',
 'Blown Away (1994)',
 'Dazed and Confused (1993)',
 'Fugitive, The (1993)',
 'Jurassic Park (1993)',
 'Mrs. Doubtfire (1993)',
 "Schindler's List (1993)",
 'So I Married an Axe Murderer (1993)',
 'Three Musketeers, The (1993)',
 'Tombstone (1993)',
 'Dances with Wolves (1990)',
 'Batman (1989)',
 'Silence of the Lambs, The (1991)',
 'Pinocchio (1940)',
 'Fargo (1996)',
 'Mission: Impossible (1996)',

In [42]:
#Get the movies to recommend by subtracting the user_watched movie list from similar movies list
recommended_movies =  set(similar_movie_list) - set(user_watched_list)

In [43]:
print(len(recommended_movies))
print(len(user_watched_list))
print(len(similar_movie_list))

24
232
130


In [44]:
#Scores of the movies in recommended movies list from similarity matrix for a selected movie
movie_name = np.array(df_movieid_title.loc[df_movieid_title.movieId == list_preferred_movieid[0],"title"])[0]
cossim_matrix.loc[movie_name,recommended_movies]

title
The Lego Movie (2014)                                                0.577350
Home (2015)                                                          0.577350
Wild, The (2006)                                                     0.632456
Shrek (2001)                                                         0.577350
Moana (2016)                                                         0.632456
Shrek the Third (2007)                                               0.632456
Toy Story 2 (1999)                                                   0.632456
Adventures of Rocky and Bullwinkle, The (2000)                       0.632456
Gnomeo & Juliet (2011)                                               0.577350
Asterix and the Vikings (Astérix et les Vikings) (2006)              0.632456
The Good Dinosaur (2015)                                             0.632456
Twelve Tasks of Asterix, The (Les douze travaux d'Astérix) (1976)    0.577350
Ant Bully, The (2006)                                     

<a id = 1> Function: user_preference_withdate</a>

In [45]:
#function to get top_n movies for a particular user after sorting the movies on recency based on timestamp
def user_preference_withdate(userid, top_n =5):
    #store the highest and second highest rated movie list (including timestamp) for a particular user
    movieid_5 = df_rating_date.loc[(df_rating_date['userId'] == userid) & (df_rating_date['rating'] == highest),["movieId","timestamp"]]
    movieid_4 = df_rating_date.loc[(df_rating_date['userId'] == userid) & (df_rating_date['rating'] == second_highest),["movieId","timestamp"]]
    
    #Sorting based on timestamp, most recent dates come first
    movie_5 = movieid_5.sort_values("timestamp",ascending = False)
    movie_4 = movieid_4.sort_values("timestamp",ascending = False)
    
    #If the length of the highest rated movie list is more than top_n , then choose top_n recent movies watched
    if len(movieid_5) > top_n:
        return(movie_5.iloc[0:top_n,0].tolist())
    #Else If the length of the highest rated movie list plus the second highest rated movie list
    #is more than top_n , then choose all the movies in highest list and the remaining movies from second highest list 
    #based on recency
    elif (len(movieid_5) + len(movieid_4)) > top_n:
        return(movie_5.iloc[:,0].tolist() + movie_4.iloc[0:top_n-len(movieid_5),0].tolist())
    #Else return all movies from highest rated and second highest rated list 
    elif (len(movieid_5) + len(movieid_4)) < top_n:
        return(movie_5.iloc[:,0].tolist() + movie_4.iloc[:,0].tolist())
    else:
        print('Not enough high ratings to determine user preference!')
        return(None)

In [46]:
user_preference_withdate(1)

[47, 2137, 2571, 2529, 2470]

<a id = "13">**Collaborative Filtering Recommender System** </a>

### Collaborative filtering in this notebook uses mainly three functions:
1. [get_similar_user](#0) :This function takes as input a userid say UserA and a list of rejected users, and returns the userid of the most similar user to the UserA.


2. [return_rating_similar_user](#1): This function takes as input the similar userid (we get this from get_similar_user function),the user id of current user (UserA) and the movieid and returns the rating given by the similar user for the movieid


3. [predict_rating](#2) : This is the main function, which uses the functions get_similar_user and return_rating_similar_user, to predict the rating for a given movieid and userid

Approach Taken:
    
    1.For a given user and movieid, predict the rating which the user might give for the movie based on 
    ratings given by users, similar to this user.
    
    2.If the similar user you found , has not rated for this movie, find the next most similar user and 
    so on till you find a rating for the movie and can predict the same for the current user.
    
    3.If the current user has already rated for the movie, then the same rating is returned.
    
    4.Based on these predicted ratings you can recommend which movies to watch next to the user.
    

In [1]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

### Read Data

In [2]:
df_ratings = pd.read_csv('ratings.csv')

In [3]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
#creating user_movie_matrix 
#initialize the data frame with all the userids
user_movie_matrix = pd.DataFrame({'userId':df_ratings['userId'].unique()})

In [5]:
#Creating a unique list of all movie ids
list_movies = list(df_ratings['movieId'].sort_values().unique())

In [6]:
#Creating a unique list of all the users
list_users = list(df_ratings['userId'].sort_values().unique())

In [7]:
#For each of the movieids, create a column in the user_movie_matrix and initialize with rating 0.0
for movie in list_movies:
    user_movie_matrix[movie] = 0.0

In [8]:
user_movie_matrix.head()

,userId,1,2,3,4,5,6,7,8,9,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#Set the index for the user_movie_matrix as userid
user_movie_matrix.index = user_movie_matrix['userId']

In [10]:
#Fill the user_movie matrix with ratings corresponding to a particular user and movieid, ratings are taken from file ratings.csv
for index,row in user_movie_matrix.iterrows():
    #tmp stores list of all movieids for a particular user
    tmp = list(df_ratings.loc[df_ratings['userId'] == index,'movieId'])
    #print(index)
    # for each of the movieids, get the corresponding rating from df_ratings dataframe and fill the user_movie_matrix
    for movie in tmp:
        user_movie_matrix.loc[index,movie] = df_ratings.loc[(df_ratings['userId'] == index) & (df_ratings['movieId'] == movie),'rating'].item()
    

In [11]:
#Storing a copy of the original user_movie matrix with filled ratings for later use
user_movie_matrix_orig = user_movie_matrix.copy()

In [12]:
#Remove userId column from the user_movie_matrix
user_movie_matrix.drop('userId',axis = 1)

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Calculating Similarity using centered cosine similarity

In [13]:
#For calculating centered cosine similarity, iterate through each of the rows of user_movie_matrix and calculate the average
#ratings for each row, by taking a sum of all the non-zero ratings for all movies watched by a user and 
#dividing with the number of movies watched by user
for index,row in user_movie_matrix.iterrows():
    #print(index)
    #initialize count to calculate the number of ratings and sum_ratings to find sum of ratings for a user
    count = 0
    sum_ratings = 0
    #For each of the movies, if the rating is non-zero, sum the ratings
    for movie in list_movies:
        if (user_movie_matrix.loc[index,movie] != 0) or (user_movie_matrix.loc[index,movie] != 0.0):
            count +=1
            sum_ratings += user_movie_matrix.loc[index,movie]
    #If user has provided atleast one rating, then store the Average calculated in column Average
    if count != 0:
        #calculating average and storing the value
        user_movie_matrix.loc[index,'Average'] = round(sum_ratings/count,3)
        #For each of the movie ratings, subtract the average rating from the corresponding movie rating,
        #to get mean centered ratings
        for movie in list_movies:
            if (user_movie_matrix.loc[index,movie] != 0) or (user_movie_matrix.loc[index,movie] != 0.0):
                user_movie_matrix.loc[index,movie] = user_movie_matrix.loc[index,'Average'] - user_movie_matrix.loc[index,movie]
    else:
        user_movie_matrix.loc[index,'Average'] = 0.0

In [14]:
user_movie_matrix.head(5)

,userId,1,2,3,4,5,6,7,8,9,...,193567,193571,193573,193579,193581,193583,193585,193587,193609,Average
userId,,,,,,,,,,,,,,,,,,,,,
1,1,0.366,0.0,0.366,0.0,0.0,0.366,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.366
2,2,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.948
3,3,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.436
4,4,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.556
5,5,-0.364,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.636


In [15]:
#test_matrix is a copy of the mean centered user_movie_matrix
test_matrix = user_movie_matrix.copy()

In [16]:
#Drop columns Average and userid as they are not required to calculate cosine similarity
test_matrix = test_matrix.drop(['Average','userId'],axis = 1)

In [17]:
test_matrix.head()

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,0.366,0.0,0.366,0.0,0.0,0.366,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-0.364,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#Create user - user cosine similarity matrix
cossim_matrix = pd.DataFrame(cosine_similarity(test_matrix))

In [19]:
#set the index and column names as userid
cossim_matrix.index = test_matrix.index
cossim_matrix.columns = test_matrix.index

In [20]:
cossim_matrix

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.001272,0.000549,0.048419,0.021856,-0.045462,-0.006158,0.047005,0.019534,-0.008777,...,0.018116,-0.017116,-0.015222,-0.037049,-0.029106,0.012028,0.055255,0.075224,-0.025713,0.010969
2,0.001272,1.000000,0.000000,-0.017176,0.021777,-0.021033,-0.011126,-0.048087,0.000000,0.003003,...,-0.050520,-0.031564,-0.001688,0.000000,0.000000,0.006228,-0.020499,-0.005989,-0.060096,0.025028
3,0.000549,0.000000,1.000000,-0.011257,-0.031549,0.004811,0.000000,-0.032483,0.000000,0.000000,...,-0.004887,-0.016107,0.017749,0.000000,-0.001428,-0.037299,-0.007790,-0.013005,0.000000,0.019546
4,0.048419,-0.017176,-0.011257,1.000000,-0.029632,0.013965,0.058085,0.002033,-0.005878,0.051613,...,-0.037682,0.063121,0.027607,-0.013790,0.040034,0.020551,0.014640,-0.037590,-0.017897,-0.001021
5,0.021856,0.021777,-0.031549,-0.029632,1.000000,0.009153,0.010126,-0.012229,0.000000,-0.033176,...,0.015967,0.012557,0.027070,0.012468,-0.036239,0.026334,0.031899,-0.001759,0.093886,-0.000265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.012028,0.006228,-0.037299,0.020551,0.026334,-0.009118,0.028354,0.022284,0.031649,-0.039967,...,0.053652,0.016400,0.098014,0.061076,0.019695,1.000000,0.017930,0.056706,0.038412,0.075510
607,0.055255,-0.020499,-0.007790,0.014640,0.031899,0.045501,0.030988,0.048850,-0.012158,-0.017664,...,0.049028,0.038207,0.049316,0.002359,-0.029373,0.017930,1.000000,0.044518,0.019076,0.021861
608,0.075224,-0.005989,-0.013005,-0.037590,-0.001759,0.021769,0.028450,0.071752,0.032784,-0.052024,...,0.069146,0.051394,0.012794,0.006321,-0.007962,0.056706,0.044518,1.000000,0.050708,0.054383


In [21]:
#this list stores the users who show maximum similarity to a particular user say UserA, but hasnt rated a particular movie, 
#so we cannot use these users for predicting the rating for UserA
list_rejected_users = []

<a id = 2> Function: predict_rating</a>

In [22]:
#for k nearest neighbour
#for a given userid and movieid,predict the rating(if user hasnt watched the movie)
#If user has already watched the movie and rated, the rating given by the user is returned
def predict_rating(userid,movieid):
    #If the rating given by a user is zero for a particular movie, that means the user has not watched the movie yet,
    #So we use the function get_similar_user to get the user most similar to the partcular user, 
    #and we check if that user has rated for the movie, if yes, we will predict the same rating
    #If the most similar user has not rated for the movie, then we append this user to the 
    #rejected user list and we find the next most similar user to the current user.
    #This process is repeated till we predict rating for a movie
    if user_movie_matrix_orig.loc[userid,movieid] == 0.0:
        #get_similar_user function called to get the most similar user for the current user based on cosine similarity score
        similar_user = get_similar_user(userid,list_rejected_users)
        #return_rating_similar_user function is called to get the rating for the movie given by the similar user
        #this function is a recursive function, which recursively runs to predict rating for a movie 
        #by similar users for current user
        rating = return_rating_similar_user(similar_user,userid,movieid)
        
        #If rating has been returned , then that is the predicted rating , else we can say 
        #none of the users watched and rated for the movie
        if rating !=  None:
            return(rating)
        else:
            print("No user has watched this movie!")
            return(None)
    else:
        #This condition says that for the userid and movieid passed, the user has rated for the movie, 
        #and that rating is returned
        print("Current user has rated for this movie:")
        rating_current_user = user_movie_matrix_orig.loc[userid,movieid]
        return(rating_current_user)
        
        
    

<a id = 1>Function: return_rating_similar_user</a>

In [23]:
#This function takes parameters similar userid, the current userid and the movieid and
#returns the rating given by the similar userid for the movieid
def return_rating_similar_user(userid,orig_user,movieid):
    #Get the rating given by the similar userid for the movieid from the original ratings matrix
    rating =  user_movie_matrix_orig.loc[userid,movieid]
    #count is used to break the while loop after a number of tries
    count = 0
    #If the rating given by the similar userid is zero, 
    #then we need to recursively call the function to get the rating by the next similar user
    while((rating == 0.0) & (count < len(list_users))):
        count += 1
        #the similar user id is updated into the list of rejected users, as the user hasnt rated for the movie
        list_rejected_users.append(userid)
        #print("Given similar user hasnt watched this movie, getting next similar user")
        next_similar_user = get_similar_user(orig_user,list_rejected_users)
        #print(list_rejected_users)
        #recursive function calling
        rating = return_rating_similar_user(next_similar_user,orig_user,movieid)
    #clearing the list of rejected users for the next run
    list_rejected_users.clear()
    return(rating)

<a id = 0>Function: get_similar_user </a>

In [24]:
#Get similar user function takes parameters userid and the list of rejected users we dont want to consider similarity 
def get_similar_user(userid,list_rejected_users):
    #df_tmp stores the corresponding row for the userid from the user-user cosine similarity matrix
    df_tmp = pd.DataFrame(cossim_matrix.loc[userid,:])
    #max_sim stores the maximum similarity of a user with another user and max_userid stores the corresponding 
    #most similar userid
    max_sim = 0
    max_userid = 0
    #new_list is a list of all unique users after removing the current userid and the list of rejected users
    #we want to avoid the checking of similarity value between the userid with itself(which gives a value of 1)
    new_list = list_users.copy()
    new_list.remove(userid)
    if len(list_rejected_users) !=0:
        for i in list_rejected_users:
            new_list.remove(i)
    #for each of the users in list, find the userid with maximum similarity value and return that userid
    for i in new_list:
        if df_tmp.loc[i,userid].item() > max_sim:
            max_sim = df_tmp.loc[i,userid].item()
            max_userid = i
    return(max_userid)   

### Function Call

In [25]:
#function predict_rating takes parameters userid and movieid and returned the predicted rating for the movie
predict_rating(114,2)

3.5